
# 1 - Imports and defining functions

In [307]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2- Loading and processing a mesh

### Basic Mesh methods

A TriMesh class can be created from a path (to a .off or a .obj file) or simply an array of vertices and an optional array of faces.

The mesh can be centered, area-normalized, rotated or translated when loading.


Vertices and faces are stored in the 'vertlist' and 'facelist' attributes. One can also use 'mesh.vertices' and 'mesh.faces' to access them. While these notations can feel non-intuitive they result in clearer functions as it avoids expressions of the form ```mesh.vertices - vertices```.

A TriMesh class possess multiple attributes like edges, per-face area, per-vertex area, per-face normals, per-vertex normals, ...

In [308]:
mesh1 = TriMesh('data/1/target.off', area_normalize=True, center=False)
mesh2 = TriMesh(mesh1.vertlist, mesh1.facelist)

In [309]:
# Attributes are computed on the fly and cached
edges = mesh1.edges

area = mesh1.area

face_areas = mesh1.face_areas
vertex_areas = mesh1.vertex_areas
face_normals = mesh1.normals

# AREA WEIGHTED VERTEX NORMALS
vertex_normals_a = mesh1.vertex_normals

# UNIFORM WEIGHTED VERTEX NORMALS
mesh1.set_vertex_normal_weighting('uniform')
vertex_normals_u = mesh1.vertex_normals

### Geodesics

We propose three versions to compute geodesics :
- Heat method - based on [potpourri3d](https://github.com/nmwsharp/potpourri3d) using robust laplacian (recommended)
- Heat method - pure python implementation from pyFM (not robust but control on the whole code)
- Dijkstra

In [310]:
# Geodesic distance from a given index
# Set robust to False to obtain result from the Python implementation
dists = mesh1.geod_from(1000, robust=True)

In [311]:
S1_geod = mesh1.get_geodesic(verbose=True)

100%|██████████| 15771/15771 [02:19<00:00, 113.26it/s]


### Laplacian and functions

The spectrum of the LBO can be computed easily.

Eigenvalues and eigenvectors are stored in the ```mesh.eigenvalues``` and ```mesh.eigenvectors``` attributes.

Gradient and divergence can be computed using the associated methods. Using the ```mesh.project``` and ```mesh.unproject``` functions allows to switch between seeing a function in the LBO basis or on the complete shape.

The squared $L^2$ norm and $H^1_0$ norm can be computed via the ```mesh.l2_sqnorm``` and ```mesh.h1_sqnorm``` methods.

In [312]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=100, intrinsic=False, verbose=True);

Computing 100 eigenvectors
	Done in 1.79 s


In [313]:
# set up a figure twice as wide as it is tall
d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, 0])
for i in range(1,10):
    d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)

#d.save("myplottest")

# 3 - Computing the functional map

**Loading data (1/2)**

In [314]:
mesh1 = TriMesh('data/1/target.off')

**Loading data (2/2)**

In [315]:
#♦mesh2 = TriMesh('data/2022-11-23-CT/1_cropped_scaled_freeze_translated.off')
mesh2 = TriMesh('data/1/source.off')
#mesh2 = TriMesh('data/2022-11-23-CT/CT_cleaned_03_removed_isolated_decimated.off')

**Displaying data**

In [333]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])
mp.subplot(mesh2.vertlist, mesh2.facelist, None, s=[2, 2, 1], data=d)



Mesh 1 : 15771 vertices, 26827 faces
Mesh 2 : 9839 vertices, 19078 faces


**Add correspondences**

In [355]:
landmarks = np.loadtxt('data/1/landmarks_vertices.txt',dtype=int)[:5]  # loading 5 landmarks
display(landmarks)

landmarks_plot_CT = TriMesh(mesh1.vertlist[landmarks[:,0]])
landmarks_plot_RGBD = TriMesh(mesh2.vertlist[landmarks[:,1]])

#landmarks_plot_CT = TriMesh(mesh1.facelist[landmarks[:,0]])
#landmarks_plot_RGBD = TriMesh(mesh2.facelist[landmarks[:,1]])

print('Retrieved matching faces from CT & RGBD pointclouds')

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])

p1 = mp.plot(mesh1.vertlist, mesh1.facelist, None)
p2 = mp.plot(mesh2.vertlist, mesh2.facelist, None)


color = c=np.random.rand(*landmarks_plot_CT.vertlist.shape)
p1.add_points(landmarks_plot_CT.vertlist, c = color, shading={"point_size": 30})
p2.add_points(landmarks_plot_RGBD.vertlist, c = color, shading={"point_size": 30})

#i = 0
#for vert in landmarks_plot_CT.vertlist:
#    color = c=np.random.rand(3)
#    p1.add_points(landmarks_plot_CT.vertlist[i], shading={"point_size": 15, "point_color": color})
#    p2.add_points(landmarks_plot_RGBD.vertlist[i], shading={"point_size": 15, "point_color": color})
#    i = i+1

#p = mp.plot(landmarks_plot_CT.vertlist, landmarks_plot_CT.facelist, c = None, shading={"point_size": 15})
#mp.subplot(landmarks_plot_RGBD.vertlist, landmarks_plot_RGBD.facelist, None, s=[2, 2, 3], data=d, shading={"point_size": 15})

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)



array([[ 1839,  1425],
       [ 4670,  3233],
       [ 7828,  5203],
       [10758,  7214],
       [13918,  9267]])

Retrieved matching faces from CT & RGBD pointclouds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.4163646…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(44.716138…

1

**Computing descriptors**

In [317]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    #'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'HKS',  # WKS or HKS
}


model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 5.48 s
Computing 200 eigenvectors
	Done in 3.98 s

Computing descriptors
	Normalizing descriptors

	20 out of 100 possible descriptors kept


**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [318]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 1.4e+01

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 20 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 1877, nit : 1750, warnflag : 0
	Done in 3.46 seconds


**Visualizing the associated point to point map**

In [319]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 4 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [320]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

100%|██████████| 10/10 [00:39<00:00,  3.91s/it]


**Zoomout**

In [321]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

100%|██████████| 15/15 [01:03<00:00,  4.23s/it]


(50, 50)
